<a href="https://colab.research.google.com/github/StarkArk/classifying_credit_risk/blob/main/credit_risk_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
lending_df = pd.read_csv("lending_data.csv")

# Review the DataFrame
display(lending_df.sample(5), lending_df.head(),lending_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77536 entries, 0 to 77535
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   loan_size         77536 non-null  float64
 1   interest_rate     77536 non-null  float64
 2   borrower_income   77536 non-null  int64  
 3   debt_to_income    77536 non-null  float64
 4   num_of_accounts   77536 non-null  int64  
 5   derogatory_marks  77536 non-null  int64  
 6   total_debt        77536 non-null  int64  
 7   loan_status       77536 non-null  int64  
dtypes: float64(3), int64(5)
memory usage: 4.7 MB


,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
51903,8500.0,6.735,44000,0.318182,3,0,14000,0
15446,10100.0,7.432,50500,0.405941,4,1,20500,0
45184,10300.0,7.499,51200,0.414062,4,1,21200,0
6313,9600.0,7.198,48300,0.378882,4,0,18300,0
5161,8300.0,6.668,43300,0.307159,3,0,13300,0


,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


None

In [3]:
lending_df['loan_status'].value_counts()    # 2500 defaults

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [4]:
lending_df.columns

Index(['loan_size', 'interest_rate', 'borrower_income', 'debt_to_income',
       'num_of_accounts', 'derogatory_marks', 'total_debt', 'loan_status'],
      dtype='object')

In [5]:
# Separate the data into labels and features
# Separate the y variable, the labels
y = lending_df['loan_status']

# Separate the X variable, the features
X = lending_df[['loan_size', 'interest_rate', 'borrower_income', 'debt_to_income',
       'num_of_accounts', 'derogatory_marks', 'total_debt']]

In [8]:
# Review the y variable Series
np.array(y.sample(1000))

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [9]:
# Review the X variable DataFrame
X

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000
...,...,...,...,...,...,...,...
77531,19100.0,11.261,86600,0.653580,12,2,56600
77532,17700.0,10.662,80900,0.629172,11,2,50900
77533,17600.0,10.595,80300,0.626401,11,2,50300
77534,16300.0,10.068,75300,0.601594,10,2,45300


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [10]:
# Check the balance of our target values
lending_df['loan_status'].value_counts()    # 2500 defaults

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [11]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [12]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(random_state=1)
classifier

# Fit the model using training data
classifier.fit(X_train, y_train)


LogisticRegression(random_state=1)

### Validate model using training and test data

In [13]:
# Score the model
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.9921240885954051
Testing Data Score: 0.9918489475856377


### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [14]:
# Make a prediction using the testing data
logistic_predictions = classifier.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [15]:
# Print the balanced_accuracy score of the model
b_accuracy = balanced_accuracy_score(y_test, logistic_predictions)
print(f'Balanced Accuracy: {round(b_accuracy*100, 2)}%')

Balanced Accuracy: 95.2%


In [16]:
# Generate a confusion matrix for the model
samp_conf = confusion_matrix(y_test, logistic_predictions)

print(samp_conf)
# tn, fp, fn, tp

[[18663   102]
 [   56   563]]


In [17]:
# Print the classification report for the model
samp_class = classification_report(y_test, logistic_predictions, target_names = ['healthy loans', 'high-risk loans'])

print(samp_class)

                 precision    recall  f1-score   support

  healthy loans       1.00      0.99      1.00     18765
high-risk loans       0.85      0.91      0.88       619

       accuracy                           0.99     19384
      macro avg       0.92      0.95      0.94     19384
   weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

### **Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

***The logistic regression model predictions for healthy loans are accurate with relatively few false positives(Precision 1.00) and false negatives(Recall .99). The predictions for high risk loans are less accurate with relatively more false positives(Precision .85) and false negatives(Recall .91). Ideally, we would like to reduce the false negative for high risk loans so that we find more of the high-risk loans that Banks would be best off avoiding.***

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points.

In [18]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
oversampler = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_resampled, y_resampled = oversampler.fit_resample(X_train, y_train)

In [19]:
# Count the distinct values of the resampled labels data
y_resampled.value_counts()

0    56271
1    56271
Name: loan_status, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [20]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(random_state=1)

# Fit the model using the resampled training data
classifier.fit(X_resampled, y_resampled)

# Make a prediction using the testing data
oversamp_predictions = classifier.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [21]:
# Print the balanced_accuracy score of the model
oversamp_b_accuracy = balanced_accuracy_score(y_test, oversamp_predictions)
print(f'Balanced Accuracy: {round(oversamp_b_accuracy*100, 2)}%')

Balanced Accuracy: 99.37%


In [22]:
# Generate a confusion matrix for the model
oversamp_conf = confusion_matrix(y_test, oversamp_predictions)

print(oversamp_conf)
# tn, fp, fn, tp

[[18649   116]
 [    4   615]]


In [23]:
# Print the classification report for the model
oversamp_class = classification_report(y_test, oversamp_predictions, target_names = ['healthy loans', 'high-risk loans'])

print(oversamp_class)

                 precision    recall  f1-score   support

  healthy loans       1.00      0.99      1.00     18765
high-risk loans       0.84      0.99      0.91       619

       accuracy                           0.99     19384
      macro avg       0.92      0.99      0.95     19384
   weighted avg       0.99      0.99      0.99     19384



In [28]:
# Compare the models

%%capture cap

print("---Balanced Accuracy---")
print("")
print(f"  Sampled: {round(100*b_accuracy,2)}%")
print(f"  OverSampled: {round(100*oversamp_b_accuracy, 2)}%")
print("")
print("---Confusion Martrices---")
print("")
print("  ---Sampled---")
print("")
print(samp_conf)
print("")
print("  ---OverSampled---")
print("")
print(oversamp_conf)
print("")
print("---Classification Tables---")
print("")
print("  ---Sampled---")
print("")
print(samp_class)
print("")
print("  ---OverSampled---")
print("")
print(oversamp_class)


with open('output.txt', 'w') as file:
    file.write(cap.stdout)

### Step 4: Answer the following question

### **Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

***Oversampling significantly reduced the # of false negatives, from 56 down to 4. Successfully detecting more high-risk loans compared to the less balanced original training sample (615 vs 563). False positives, loans eroneously labeled 'high-risk', did increase. This is acceptable given that a bank should be more focused on avoiding/reducing bad loans rather than picking up a marginal number of 'healthy' loans.***